# Agent Demo with Simplified Connection Management

This notebook demonstrates an agent using multiple tool types with simplified connection management:
1. **Local Calculator**: Mathematical expressions via MCP
2. **Brave Search**: Web search via MCP (if API key available)
3. **Think Tool**: Internal reasoning tool

In [1]:
import os
import sys
import nest_asyncio
nest_asyncio.apply()

parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)
from agents.agent import Agent, ModelConfig
from agents.tools.think import ThinkTool

# 1. Configure tools and MCP servers

In [ ]:
# Standard Python tool
think_tool = ThinkTool()

# Python MCP server
calculator_server_path = os.path.abspath(os.path.join(os.getcwd(), "tools/calculator_mcp.py"))
calculator_server = {
    "type": "stdio",
    "command": "python",
    "args": [calculator_server_path]
}
print(f"Calculator server configured: {'Yes' if calculator_server else 'No'}")


# Brave MCP server written in TypeScript

brave_api_key = os.environ.get("BRAVE_API_KEY_BASE_DATA", "")
print(f"Brave API key available: {'Yes' if brave_api_key else 'No'}")
brave_search_server = {
    "type": "stdio",
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-brave-search"],
    "env": {
        "BRAVE_API_KEY": brave_api_key, 
        "PATH": f"{os.path.dirname('npx')}:" + os.environ.get("PATH", "")
    }
}
print(f"Brave search server configured: {'Yes' if brave_search_server else 'No'}")



Calculator server configured: Yes
Brave API key available: Yes
Brave search server configured: Yes


# 2. Initialize agent with combined tools

In [3]:
# Create agent config 
system_prompt = """
You are a helpful assistant with access to:
1. Web search (brave_web_search, brave_local_search)
2. Mathematical calculator (calculate) 
3. A tool to think and reason (think)

Always use the most appropriate tool for each task.
"""


# Initialize agent with standard tools and MCP servers
agent = Agent(
    name="Multi-Tool Agent",
    system=system_prompt,
    tools=[think_tool], 
    mcp_servers=[brave_search_server, calculator_server], 
    config=ModelConfig(
        model="claude-3-7-sonnet-20250219", 
        max_tokens=4096,
        temperature=1.0
    ),
    verbose=True
)


[Multi-Tool Agent] Agent initialized


# 3. Run queries

In [4]:
# Example query
agent.run("What's the square root of the OKC population in 2022")

Loaded 3 MCP tools from 2 servers.

[Multi-Tool Agent] Received: What's the square root of the OKC population in 2022

[Multi-Tool Agent] Output: To answer this question accurately, I'll need to find the population of Oklahoma City (OKC) in 2022, then calculate its square root.

[Multi-Tool Agent] Tool call: brave_web_search(query=Oklahoma City population 2022 official census)

[Multi-Tool Agent] Tool result: Title: Oklahoma Cities by Population (2025)
Description: The most populous cities in Oklahoma are <strong>Oklahoma City with 702,767</strong>, Tulsa at 411,894, Norman with 130,046, Broken Arrow at 119,194, and Edmond with 98,103. You can get the most current Census population data for all 834 Oklahoma cities, sorted from the most populous cities in Oklahoma to the ...
URL: https://www.oklahoma-demographics.com/cities_by_population

Title: Oklahoma City city, Oklahoma - Census Bureau Profile
Description: We&#x27;re sorry but this website doesn&#x27;t work properly without JavaScri

Message(id='msg_019BwLDj3ycHkPdqWyqKHQzZ', content=[TextBlock(citations=None, text='Based on my research, the population of Oklahoma City in 2022 was approximately 694,800 people. The square root of this population number is 833.55.\n\nSo the square root of the OKC population in 2022 is 833.55.', type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=127, cache_read_input_tokens=3983, input_tokens=5, output_tokens=63))

In [5]:
await agent.run_async("How many bananas will fit in an Toyota GR86?")

Loaded 3 MCP tools from 2 servers.

[Multi-Tool Agent] Received: How many bananas will fit in an Toyota GR86?

[Multi-Tool Agent] Output: To answer this question accurately, I'll need to find information about the cargo capacity of a Toyota GR86 and the approximate size of bananas to make a reasonable estimate.

[Multi-Tool Agent] Tool call: brave_web_search(query=Toyota GR86 cargo capacity trunk space dimensions cubic feet)

[Multi-Tool Agent] Tool result: Title: Cargo space. What can it fit? : r/GR86
Description: 11 votes, 36 comments. 41K subscribers in the <strong>GR86</strong> community. For anyone interested in the <strong>Toyota</strong> <strong>GR86</strong> &amp; Subaru BRZ (&#x27;22+).
URL: https://www.reddit.com/r/GR86/comments/16xh6ii/cargo_space_what_can_it_fit/

Title: 2025 Toyota GR86 Interior, Cargo Space & Seating | U.S. News
Description: Few sports cars have large trunks, and the GR86 is no exception. It has just <strong>6.3 cubic feet</strong> of trunk space, which i

Message(id='msg_01WzSwaTgvBufqiCyLBvjurg', content=[TextBlock(citations=None, text="Based on my research, I can provide an estimate of how many bananas would fit in a Toyota GR86:\n\nThe Toyota GR86 has approximately 6.3 cubic feet of trunk space according to multiple sources. The average banana has a volume of around 130 cubic centimeters (0.0046 cubic feet).\n\nIn theory, if you could pack bananas with 100% efficiency (which isn't possible due to their shape), you could fit about 1,370 bananas in the trunk. However, real-world packing efficiency is closer to 65% due to:\n\n1. The curved shape of bananas creating unavoidable empty spaces\n2. The need to avoid crushing the bananas\n3. The irregular dimensions of the trunk space\n\nWith a more realistic 65% packing efficiency, you could fit approximately 890 bananas in the trunk of a Toyota GR86.\n\nIf you were to fold down the rear seats (which the GR86 has, although they're small), you could potentially fit a few hundred more bananas 